In [1]:
import pandas as pd
from geopy.distance import geodesic
from tqdm import tqdm
tqdm.pandas()

#Считаем количество жилой площади в домах на расстоянии не более 500м от банкомата
#Создаём общий csv с объектами инфраструктуры

In [13]:
MAX_DISTANCE_KM = 500 / 1000

In [3]:
# distance = geodesic(atm_coords, obj_coords).km #lat, long

In [4]:
df = pd.read_csv("concat_df.csv").drop("Unnamed: 0", axis='columns')
print(len(df))
df.head()

8461


,id,atm_group,address,address_rus,lat,long,target
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277


In [5]:
df_gps = df[['lat','long']].drop_duplicates().dropna()
len(df_gps)

6243

In [6]:
df_gps.head()

,lat,long
0,46.940995,142.738319
1,46.937353,142.753348
2,46.959413,142.741113
3,55.805827,37.515146
4,69.343541,88.211228


In [7]:
atm_n = 0
atm_loc = (df_gps.iloc[atm_n, 0], df_gps.iloc[atm_n, 1])
atm_loc

(46.940995, 142.738319)

In [9]:
df_obj = pd.read_csv("final_csv.csv")
df_obj.head()

,id,address,area_residential,filename,lat,lon
0,9001067,"Респ. Адыгея, г. Адыгейск, д. 888888","2,00",export-reestrmkd-01-20221101.csv,44.883943,39.190786
1,6472757,"Респ. Адыгея, г. Адыгейск, ул. Горького, д. 25","1818,00",export-reestrmkd-01-20221101.csv,44.879606,39.190019
2,6472669,"Респ. Адыгея, г. Адыгейск, ул. Горького, д. 27","1818,00",export-reestrmkd-01-20221101.csv,44.879770,39.191084
3,8928733,"Респ. Адыгея, г. Адыгейск, ул. Горького, д. 31","2729,00",export-reestrmkd-01-20221101.csv,44.880272,39.191226
4,6551241,"Респ. Адыгея, г. Адыгейск, ул. Коммунистическа...","2522,00",export-reestrmkd-01-20221101.csv,44.879207,39.182191


In [10]:
len(df_obj)

827952

In [19]:
#Оставляем только координаты
filter4 = df_obj[['lat', 'lon','area_residential']].drop_duplicates()
filter4['area_residential'] = filter4['area_residential'].str.replace(",",".").astype('float64')
filter4

,lat,lon,area_residential
0,44.883943,39.190786,2.0
1,44.879606,39.190019,1818.0
2,44.879770,39.191084,1818.0
3,44.880272,39.191226,2729.0
4,44.879207,39.182191,2522.0
...,...,...,...
827947,44.596348,33.567618,71.3
827948,44.595917,33.568670,71.3
827949,44.595396,33.568897,72.0
827950,44.535673,33.620196,387.9


In [14]:
x1, y1 = 0, 0
x2, y2 = 0.01, 0


dist = geodesic((x1, y1), (x2, y2)).km

dist, dist > MAX_DISTANCE_KM

(1.1057427583286865, True)

In [20]:
%%time

#Возвращаем значение жилой площади на расстоянии не более 500м от входящих координат (банкомата)
def find_distance(row, atm_loc):
    if all(pd.isna(row)):
        return None
    obj_loc = (row['lat'], row['lon'])
    distance = geodesic(atm_loc, obj_loc).km
    return 0 if distance > MAX_DISTANCE_KM else row['area_residential']

#Функция отбрасывает все объекты на расстоянии примерно 1км и больше
def value_checker(x, y):
    return abs(x - y) <= 0.01


def find_atm_distance(row):    
    atm_loc = (row['lat'], row['long'])
    filter_x = filter4[filter4['lat'].apply(value_checker, args=(atm_loc[0],))]
    filter_r = filter_x[filter_x['lon'].apply(value_checker, args=(atm_loc[1],))]
    distances = filter_r.reset_index().apply(find_distance, args=(atm_loc,), axis=1)
    return sum(distances)

sum_col = df_gps.progress_apply(find_atm_distance, axis=1)
sum_col

100%|█████████████████████████████████████████████████████████████████████| 6243/6243 [47:19<00:00,  2.20it/s]

CPU times: user 47min 4s, sys: 18.1 s, total: 47min 22s
Wall time: 47min 19s


0       160311.25
1       144069.25
2       105844.43
3       363488.80
4       312643.81
          ...    
8454    120500.20
8455    278492.11
8456    460280.50
8457     36062.98
8460     94830.70
Length: 6243, dtype: float64

In [21]:
df_gps['dist'] = sum_col
df_gps.to_csv('coords_dist_count_area.csv', index=False)

In [22]:
def check_if(row):
    x, y = row['lat'], row['long']
    check_against = df_gps[(df_gps['lat'] == x) & (df_gps['long'] == y)]['dist'].tolist()
    return check_against[0] if check_against else None

df['dist'] = df.apply(check_if, axis=1)
df

,id,atm_group,address,address_rus,lat,long,target,dist
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500,160311.25
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710,144069.25
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540,105844.43
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035,363488.80
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277,312643.81
...,...,...,...,...,...,...,...,...
8456,7805.0,5478.0,"D. 12, PROEZD ZAREVYI MOSKVA G","Заревый проезд, 12, Москва, Россия, 127282",55.886488,37.656829,NaN,460280.50
8457,7395.0,5478.0,"D. 2B, UL. MIRA SOCHI G","улица Мира, 4, Адлерский район, Сочи, Краснода...",43.438310,39.912211,NaN,36062.98
8458,3339.0,8083.0,"KARMANITSKIY, 9 Moskva","Карманицкий переулок, 9, Москва, Россия, 119002",55.748275,37.583571,NaN,375735.35
8459,675.0,1942.0,3A SHARAPOVSKIY WAY MYTISHCHI,"Шараповская улица, 1, Мытищи, Московская облас...",55.910992,37.748466,NaN,177692.24


In [23]:
df.to_csv('df_dist_count_area.csv', index=False)

In [ ]:
# Объединение признаков

In [ ]:
df_area = pd.read_csv('df_dist_count_area.csv').rename(columns={'dist':'area'})
df_dist = pd.read_csv('df_dist_count.csv')
df_dist['area'] = df_area['area']
df_dist.to_csv('df_dist_united.csv', index=False)